In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import tensorflow as tf
from tqdm import tqdm
from glob import glob
from google.colab import drive

import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/deep-voice/data/'

Mounted at /content/drive
/content/drive/MyDrive/deep-voice/data


In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

In [ ]:
# 저장된 데이터 불러오기
X_train_mfcc = np.load('X_train_mfcc.npy')
X_val_mfcc = np.load('X_val_mfcc.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense
from tensorflow_addons.layers import GroupNormalization

model = Sequential()

model.add(Bidirectional(LSTM(512, input_shape=(X_train_mfcc.shape[1], 1))))
model.add(GroupNormalization())
model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(
    monitor='val_loss',  # 관찰할 값
    patience=3,          # 성능 향상이 없는 에폭을 몇 번이나 허용할 것인가
    verbose=1,           # 진행 상황을 출력
    mode='min',          # 'min' 모드는 관찰하는 값의 감소가 멈추면 훈련을 중단
    restore_best_weights=True  # 가장 좋은 모델의 가중치를 복원
)

# 모델 훈련
history = model.fit(
    X_train_mfcc, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val_mfcc, y_val),
    callbacks=[early_stopping]  # 콜백 리스트에 early_stopping 추가
)

Epoch 1/10
40/40 [==============================] - 390s 10s/step - loss: 0.1210 - accuracy: 0.9578 - val_loss: 0.0161 - val_accuracy: 0.9937
Epoch 2/10
40/40 [==============================] - 376s 9s/step - loss: 7.0005e-04 - accuracy: 1.0000 - val_loss: 0.0227 - val_accuracy: 0.9937
Epoch 3/10
40/40 [==============================] - 378s 9s/step - loss: 8.4539e-04 - accuracy: 1.0000 - val_loss: 0.0216 - val_accuracy: 0.9937
Epoch 4/10
40/40 [==============================] - 373s 9s/step - loss: 0.0014 - accuracy: 0.9992 - val_loss: 0.0089 - val_accuracy: 0.9969
Epoch 5/10
40/40 [==============================] - 375s 9s/step - loss: 3.0970e-04 - accuracy: 1.0000 - val_loss: 0.0112 - val_accuracy: 0.9969
Epoch 6/10
40/40 [==============================] - 373s 9s/step - loss: 2.2241e-04 - accuracy: 1.0000 - val_loss: 0.0101 - val_accuracy: 0.9969
Epoch 7/10
40/40 [==============================] - 374s 9s/step - loss: 1.6721e-04 - accuracy: 1.0000 - val_loss: 0.0117 - val_accuracy: